In [1]:
!pip install transformers datasets gradio scikit-learn nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import os

print(os.listdir("/content"))


['.config', '.ipynb_checkpoints', 'enterprise-attack-v16.1-relationships.xlsx', 'enterprise-attack-v16.1-techniques.xlsx', 'enterprise-attack-v16.1-tactics.xlsx']


In [8]:
!pip install pandas openpyxl
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords


In [10]:
# Load Excel Files
relationships_df = pd.read_excel("/content/enterprise-attack-v16.1-relationships.xlsx")
tactics_df = pd.read_excel("/content/enterprise-attack-v16.1-tactics.xlsx")
techniques_df = pd.read_excel("/content/enterprise-attack-v16.1-techniques.xlsx")

# Display Sample Data
print("Relationships Data:")
display(relationships_df.head())

print("\nTactics Data:")
display(tactics_df.head())

print("\nTechniques Data:")
display(techniques_df.head())


Relationships Data:


,source ID,source name,source ref,source type,mapping type,target ID,target name,target ref,target type,mapping description,STIX ID,created,last modified
0,C0028,2015 Ukraine Electric Power Attack,campaign--46421788-b6e1-4256-b351-f8beffd1afba,campaign,attributed-to,G0034,Sandworm Team,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,group,(Citation: Andy Greenberg June 2017) (Citation...,relationship--4d407dda-944a-4974-b1c2-0a04d2c9...,27 September 2023,27 September 2023
1,C0025,2016 Ukraine Electric Power Attack,campaign--aa73efef-1418-4dbe-b43c-87a498e97234,campaign,attributed-to,G0034,Sandworm Team,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,group,(Citation: US District Court Indictment GRU Un...,relationship--90647f03-38a4-4364-a3af-53640a81...,31 March 2023,31 March 2023
2,C0034,2022 Ukraine Electric Power Attack,campaign--df8eb785-70f8-4300-b444-277ba849083d,campaign,attributed-to,G0034,Sandworm Team,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,group,(Citation: Mandiant-Sandworm-Ukraine-2022)(Cit...,relationship--d3717846-eaab-4fde-99f6-a972dec9...,27 March 2024,10 April 2024
3,C0040,APT41 DUST,campaign--add4d9de-1256-4166-83b8-57087288dced,campaign,attributed-to,G0096,APT41,intrusion-set--18854f55-ac7c-4634-bd9a-352dd07...,group,[APT41 DUST](https://attack.mitre.org/campaign...,relationship--31aebdb8-ce4d-4d31-a144-7a5c354b...,16 September 2024,16 September 2024
4,C0011,C0011,campaign--b4e5a4a9-f3be-4631-ba8f-da6ebb067fac,campaign,attributed-to,G0134,Transparent Tribe,intrusion-set--e44e0985-bc65-4a8f-b578-211c858...,group,(Citation: Cisco Talos Transparent Tribe Educa...,relationship--751e795e-7c1a-4ba1-bb20-636aed02...,22 September 2022,22 September 2022



Tactics Data:


,ID,STIX ID,name,description,url,created,last modified,domain,version
0,TA0009,x-mitre-tactic--d108ce10-2419-4cf9-a774-46161d...,Collection,The adversary is trying to gather data of inte...,https://attack.mitre.org/tactics/TA0009,17 October 2018,05 September 2024,enterprise-attack,1.1
1,TA0011,x-mitre-tactic--f72804c5-f15a-449e-a5da-2eecd1...,Command and Control,The adversary is trying to communicate with co...,https://attack.mitre.org/tactics/TA0011,17 October 2018,19 July 2019,enterprise-attack,1.0
2,TA0006,x-mitre-tactic--2558fd61-8c75-4730-94c4-11926d...,Credential Access,The adversary is trying to steal account names...,https://attack.mitre.org/tactics/TA0006,17 October 2018,19 July 2019,enterprise-attack,1.0
3,TA0005,x-mitre-tactic--78b23412-0651-46d7-a540-170a1c...,Defense Evasion,The adversary is trying to avoid being detecte...,https://attack.mitre.org/tactics/TA0005,17 October 2018,19 July 2019,enterprise-attack,1.0
4,TA0007,x-mitre-tactic--c17c5845-175e-4421-9713-829d05...,Discovery,The adversary is trying to figure out your env...,https://attack.mitre.org/tactics/TA0007,17 October 2018,19 July 2019,enterprise-attack,1.0



Techniques Data:


,ID,STIX ID,name,description,url,created,last modified,domain,version,tactics,...,is sub-technique,sub-technique of,defenses bypassed,contributors,permissions required,supports remote,system requirements,impact type,effective permissions,relationship citations
0,T1548,attack-pattern--67720091-eee3-4d2d-ae16-826456...,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,https://attack.mitre.org/techniques/T1548,30 January 2020,15 October 2024,enterprise-attack,1.4,"Defense Evasion, Privilege Escalation",...,False,NaN,NaN,NaN,"Administrator, User",NaN,NaN,NaN,NaN,"(Citation: TrendMicro RaspberryRobin 2022),(Ci..."
1,T1548.002,attack-pattern--120d5519-3098-4e1c-9191-2aa612...,Abuse Elevation Control Mechanism: Bypass User...,Adversaries may bypass UAC mechanisms to eleva...,https://attack.mitre.org/techniques/T1548/002,30 January 2020,21 April 2023,enterprise-attack,2.1,"Defense Evasion, Privilege Escalation",...,True,T1548,Windows User Account Control,Casey Smith; Stefan Kanthak,"Administrator, User",NaN,NaN,NaN,Administrator,"(Citation: Check Point Warzone Feb 2020),(Cita..."
2,T1548.004,attack-pattern--b84903f0-c7d5-435d-a69e-de47cc...,Abuse Elevation Control Mechanism: Elevated Ex...,Adversaries may leverage the <code>Authorizati...,https://attack.mitre.org/techniques/T1548/004,30 January 2020,19 October 2022,enterprise-attack,1.0,"Defense Evasion, Privilege Escalation",...,True,T1548,NaN,"Erika Noerenberg, @gutterchurl, Carbon Black; ...","Administrator, User",NaN,NaN,NaN,root,"(Citation: Carbon Black Shlayer Feb 2019),"
3,T1548.001,attack-pattern--6831414d-bb70-42b7-8030-d4e06b...,Abuse Elevation Control Mechanism: Setuid and ...,An adversary may abuse configurations where an...,https://attack.mitre.org/techniques/T1548/001,30 January 2020,15 March 2023,enterprise-attack,1.1,"Defense Evasion, Privilege Escalation",...,True,T1548,NaN,NaN,User,NaN,NaN,NaN,NaN,"(Citation: OSX Keydnap malware),(Citation: ANS..."
4,T1548.003,attack-pattern--1365fe3b-0f50-455d-b4da-266ce3...,Abuse Elevation Control Mechanism: Sudo and Su...,Adversaries may perform sudo caching and/or us...,https://attack.mitre.org/techniques/T1548/003,30 January 2020,14 March 2022,enterprise-attack,1.0,"Defense Evasion, Privilege Escalation",...,True,T1548,NaN,NaN,User,NaN,NaN,NaN,root,"(Citation: hexed osx.dok analysis 2019),(Citat..."


In [13]:
# Check column names and sample rows
print("Relationships Data Columns:", relationships_df.columns)
print("Tactics Data Columns:", tactics_df.columns)
print("Techniques Data Columns:", techniques_df.columns)

# Check for missing values
print("\nMissing Values in Relationships Data:\n", relationships_df.isnull().sum())
print("\nMissing Values in Tactics Data:\n", tactics_df.isnull().sum())
print("\nMissing Values in Techniques Data:\n", techniques_df.isnull().sum())


Relationships Data Columns: Index(['source ID', 'source name', 'source ref', 'source type', 'mapping type',
       'target ID', 'target name', 'target ref', 'target type',
       'mapping description', 'STIX ID', 'created', 'last modified'],
      dtype='object')
Tactics Data Columns: Index(['ID', 'STIX ID', 'name', 'description', 'url', 'created',
       'last modified', 'domain', 'version'],
      dtype='object')
Techniques Data Columns: Index(['ID', 'STIX ID', 'name', 'description', 'url', 'created',
       'last modified', 'domain', 'version', 'tactics', 'detection',
       'platforms', 'data sources', 'is sub-technique', 'sub-technique of',
       'defenses bypassed', 'contributors', 'permissions required',
       'supports remote', 'system requirements', 'impact type',
       'effective permissions', 'relationship citations'],
      dtype='object')

Missing Values in Relationships Data:
 source ID              2049
source name               0
source ref                0
source ty

In [20]:


# Ensure columns are cleaned (lowercase, no spaces)
relationships_df.columns = relationships_df.columns.str.strip().str.lower()
tactics_df.columns = tactics_df.columns.str.strip().str.lower()
techniques_df.columns = techniques_df.columns.str.strip().str.lower()

# Merge techniques with relationships (source ID -> techniques ID)
merged_df = relationships_df.merge(
    techniques_df,
    left_on='source id',
    right_on='id',
    how='left'
)

# Merge tactics with previous result (target ID -> tactics ID)
merged_df = merged_df.merge(
    tactics_df,
    left_on='target id',
    right_on='id',
    how='left'
)

# Keep only useful columns for extraction
merged_df = merged_df[['source id', 'target id', 'source name', 'name_x', 'name_y']]

# Rename columns for clarity
merged_df.columns = ['Technique_ID', 'Tactic_ID', 'Source_Name', 'Technique', 'Tactic']

# Display merged data sample
print("\n✅ Merged Data Sample:")
display(merged_df.head())
print("Missing Techniques:", merged_df['Technique'].isnull().sum())
print("Missing Tactics:", merged_df['Tactic'].isnull().sum())

merged_df.to_csv('/content/cleaned_attack_mapping.csv', index=False)

from google.colab import files
files.download('/content/cleaned_attack_mapping.csv')



✅ Merged Data Sample:


,Technique_ID,Tactic_ID,Source_Name,Technique,Tactic
0,C0028,G0034,2015 Ukraine Electric Power Attack,NaN,NaN
1,C0025,G0034,2016 Ukraine Electric Power Attack,NaN,NaN
2,C0034,G0034,2022 Ukraine Electric Power Attack,NaN,NaN
3,C0040,G0096,APT41 DUST,NaN,NaN
4,C0011,G0134,C0011,NaN,NaN


Missing Techniques: 18570
Missing Tactics: 18570


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
import re

# Clean text function to normalize input
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|[^A-Za-z0-9\s]', '', str(text))  # Ensure text is string and clean it
    return text.lower()

# Extract TTPs function
def extract_ttps(report, mapping_df):
    detected_ttps = []

    # Clean the report
    report = clean_text(report)

    # Iterate over mapping DataFrame
    for _, row in mapping_df.iterrows():
        # Ensure the row has a valid technique and tactic (skip NaN values)
        if pd.notna(row['technique']) and pd.notna(row['tactic']):
            if row['technique'] in report:
                detected_ttps.append((row['technique'], row['tactic']))

    return detected_ttps

# Sample Threat Report
sample_report = """
The attacker used spear-phishing emails to gain initial access.
Later, they used PowerShell scripts to maintain persistence and evade defenses.
"""

# Detect TTPs
detected_ttps = extract_ttps(sample_report, merged_df)

# Output detected TTPs
print("\n✅ Detected TTPs:")
if not detected_ttps:
    print("No TTPs detected.")
else:
    for technique, tactic in detected_ttps:
        print(f"Technique: {technique} | Tactic: {tactic}")



✅ Detected TTPs:
No TTPs detected.


In [26]:


# Extract IOCs (IP addresses, domains, and emails)
def extract_iocs(text):
    # IP Address Pattern (IPv4)
    ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'
    ips = re.findall(ip_pattern, text)

    # Domain Pattern (matches domains but not IP addresses)
    domain_pattern = r'\b(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}\b'
    domains = re.findall(domain_pattern, text)

    # Email Pattern
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(email_pattern, text)

    # Remove domains that are part of extracted emails to avoid duplication
    domains = [d for d in domains if d not in emails]

    return ips, domains, emails

# Sample Threat Report with IOCs
sample_report_with_iocs = """
The attacker used spear-phishing to gain initial access.
They connected to 203.0.113.45 and downloaded a malicious payload from badactor.org.
Phishing emails were sent from attacker@evilcorp.com.
"""

# Extract IOCs
ips, domains, emails = extract_iocs(sample_report_with_iocs)

# Display Extracted IOCs
print("\n✅ Extracted IOCs:")
print("IP Addresses:", ips)
print("Domains:", domains)
print("Emails:", emails)



✅ Extracted IOCs:
IP Addresses: ['203.0.113.45']
Domains: ['badactor.org', 'evilcorp.com']
Emails: ['attacker@evilcorp.com']


In [27]:
# Full Analysis Function: Extract both TTPs and IOCs
def analyze_report(report, mapping_df):
    # Extract TTPs
    detected_ttps = extract_ttps(report, mapping_df)

    # Extract IOCs
    ips, domains, emails = extract_iocs(report)

    # Display Results
    print("\n✅ Analysis Results:")

    # Show TTPs
    if detected_ttps:
        print("\nDetected TTPs:")
        for technique, tactic in detected_ttps:
            print(f"Technique: {technique} | Tactic: {tactic}")
    else:
        print("No TTPs detected.")

    # Show IOCs
    print("\nDetected IOCs:")
    print(f"- IP Addresses: {ips if ips else 'None'}")
    print(f"- Domains: {domains if domains else 'None'}")
    print(f"- Emails: {emails if emails else 'None'}")

# Analyze a Sample Report
new_report = """
The attacker used spear-phishing for initial access and employed PowerShell scripts.
They connected to 45.67.89.101 and downloaded a malicious payload from attackerdomain.org.
Phishing emails were sent to victims from hacker@badactor.net.
"""

analyze_report(new_report, merged_df)



✅ Analysis Results:
No TTPs detected.

Detected IOCs:
- IP Addresses: ['45.67.89.101']
- Domains: ['attackerdomain.org', 'badactor.net']
- Emails: ['hacker@badactor.net']


In [28]:


# Save detected TTPs to CSV
def save_results_to_csv(detected_ttps, ips, domains, emails, filename="detected_results.csv"):
    # Prepare the data
    results = []
    for technique, tactic in detected_ttps:
        results.append({
            "Technique": technique,
            "Tactic": tactic
        })

    for ip in ips:
        results.append({"IP Address": ip})

    for domain in domains:
        results.append({"Domain": domain})

    for email in emails:
        results.append({"Email": email})

    # Create and save DataFrame
    results_df = pd.DataFrame(results)
    results_df.to_csv(filename, index=False)
    print(f"\n✅ Results saved to {filename}")

# Example Usage
ips, domains, emails = extract_iocs(new_report)
save_results_to_csv(detected_ttps, ips, domains, emails)

# Download in Google Colab
from google.colab import files
files.download("detected_results.csv")



✅ Results saved to detected_results.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
!pip install gradio


In [30]:
import gradio as gr

# Gradio Prediction Function
def analyze_with_ui(input_text):
    # Run TTP and IOC extraction
    detected_ttps = extract_ttps(input_text, merged_df)
    ips, domains, emails = extract_iocs(input_text)

    # Format output
    output = "✅ **Analysis Results:**\n\n"

    # Add TTPs to the output
    if detected_ttps:
        output += "**Detected TTPs:**\n"
        for technique, tactic in detected_ttps:
            output += f"- Technique: {technique} | Tactic: {tactic}\n"
    else:
        output += "**No TTPs detected.**\n"

    # Add IOCs to the output
    output += "\n**Detected IOCs:**\n"
    output += f"- IP Addresses: {', '.join(ips) if ips else 'None'}\n"
    output += f"- Domains: {', '.join(domains) if domains else 'None'}\n"
    output += f"- Emails: {', '.join(emails) if emails else 'None'}\n"

    return output

# Launch Gradio Interface
iface = gr.Interface(fn=analyze_with_ui, inputs="text", outputs="text", title="ATT&CK TTP & IOC Extractor")
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://23d5965ed6acce202b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [31]:
from sklearn.metrics import classification_report

# Example true and predicted labels
y_true = ['T1566', 'T1059']    # Actual Techniques (from MITRE ATT&CK IDs)
y_pred = ['T1566', 'T1059']    # Predicted Techniques

print("\n✅ Evaluation Metrics:")
print(classification_report(y_true, y_pred))



✅ Evaluation Metrics:
              precision    recall  f1-score   support

       T1059       1.00      1.00      1.00         1
       T1566       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

